In [20]:
from transformers import BertForQuestionAnswering, BertTokenizer
import PyPDF2
import torch

def extract_text_from_pdf(pdf_file_path):
    try:
        with open(pdf_file_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
            return text
    except FileNotFoundError:
        return "PDF file not found."
    except PyPDF2.utils.PdfReadError:
        return "Unable to read PDF file."
    except Exception as e:
        return str(e)

def answer_question_about_pdf(question, pdf_file_path):
        pdf_text = extract_text_from_pdf(pdf_file_path)
        
        tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
        model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
        
        
        inputs = tokenizer(question, pdf_text, add_special_tokens=True, return_tensors='pt')
        outputs = model(**inputs)
    
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

        answer_start = torch.argmax(start_logits)
        answer_end = torch.argmax(end_logits) + 1

        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
        return answer

pdf_file_path = "sample.pdf"
question = "How many categories solar system divided?"
answer = answer_question_about_pdf(question, pdf_file_path)
print(answer)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


two regions
